In [2]:
import pandas as pd
import numpy as np
import requests
import json
from config import api_key
from pprint import pprint

In [3]:
base_url = "https://api.nytimes.com/svc/books/v3"
tail = f"?api-key={api_key}"

In [4]:
weeks_in_bc = ['current','2020-09-21','2020-09-14','2020-09-07','2020-08-31','2020-08-24', '2020-08-17', '2020-08-10', '2020-08-03', '2020-07-27', '2020-07-20','2020-07-13','2020-07-06','2020-06-29']
#things we want: Title, Author, ISBN, Number of Weeks on NYT Best Seller List,Rating on Kaggle

In [5]:
#Variables for lists/API call
result_count=np.arange(15)
list_name=[]
publish_date=[]
title_name=[]
author_name=[]
weeks_on_list=[]
#not used in df still figuring out
isbns=[]

In [6]:
for week in weeks_in_bc:
    response = requests.get(f"{base_url}/lists/{week}/hardcover-fiction.json{tail}")
    data=response.json()
    for i in result_count:
        listname = data['results']['list_name']
        list_name.append(listname)
        pub_date = data['results']['bestsellers_date']
        publish_date.append(pub_date)
        title = data['results']['books'][i]['title']
        title_name.append(title)
        author = data['results']['books'][i]['author']
        author_name.append(author)
        weeks = data['results']['books'][i]['weeks_on_list']
        weeks_on_list.append(weeks)
        isbnlist = data['results']['books'][i]['isbns']
        isbns.append(isbnlist)

pprint(data)

In [9]:
hc_fiction_df=pd.DataFrame({
    "List": list_name,
    "List_Date": publish_date,
    "Title": title_name,
    "Author": author_name,
    "Weeks_On_List": weeks_on_list 
})

In [10]:
hc_fiction_df.head()

,List,List_Date,Title,Author,Weeks_On_List
0,Hardcover Fiction,2020-09-19,THE EVENING AND THE MORNING,Ken Follett,1
1,Hardcover Fiction,2020-09-19,TROUBLED BLOOD,Robert Galbraith,1
2,Hardcover Fiction,2020-09-19,VINCE FLYNN: TOTAL POWER,Kyle Mills,1
3,Hardcover Fiction,2020-09-19,TO SLEEP IN A SEA OF STARS,Christopher Paolini,1
4,Hardcover Fiction,2020-09-19,THE VANISHING HALF,Brit Bennett,16


In [11]:
hc_fiction_df["Title"].nunique()

65

In [12]:
hc_titles = list(hc_fiction_df["Title"].unique())

In [13]:
#Some pbj: I need to make a new df with only the entries for the 65 titles with thier max weeks on the list...
#trying groupby title pls hold... this did not work
#trying sort and drop dupes... THIS WORKED

In [14]:
test_df = hc_fiction_df.copy().sort_values(by=["Title","Weeks_On_List"], ascending=False)

In [15]:
only_hc_titles_df = test_df.copy().drop_duplicates(subset="Title", keep="first", inplace=False)

In [16]:
only_hc_titles_df = only_hc_titles_df.sort_values(by="Weeks_On_List", ascending=False)

In [17]:
only_hc_titles_df.head()

,List,List_Date,Title,Author,Weeks_On_List
5,Hardcover Fiction,2020-09-19,WHERE THE CRAWDADS SING,Delia Owens,107
55,Hardcover Fiction,2020-08-29,THE SILENT PATIENT,Alex Michaelides,56
27,Hardcover Fiction,2020-09-12,AMERICAN DIRT,Jeanine Cummins,34
59,Hardcover Fiction,2020-08-29,IF IT BLEEDS,Stephen King,18
72,Hardcover Fiction,2020-08-22,CAMINO WINDS,John Grisham,17


In [18]:
counts = pd.DataFrame(hc_fiction_df["Title"].value_counts())
counts

,Title
WHERE THE CRAWDADS SING,14
THE VANISHING HALF,14
THE GUEST LIST,14
AMERICAN DIRT,13
28 SUMMERS,11
...,...
CHOPPY WATER,1
THE DARKEST EVENING,1
FRIENDS AND STRANGERS,1
DARK SONG,1


In [19]:
#have to take a break, daily limit reached. oops